user_collab_filtering_v1
Not very good but a model for things to come

In [12]:
from src.pipeline.etl import ETL

Collecting Data:

rating = 1 for every shiur in database. Can make something more advanced later.

In [13]:
db = ETL()
df = db.get_bookmarks_df()
df['rating'] = 1
df.shape


2024-07-01 15:37:33,523 - root - INFO - ETL instance created
2024-07-01 15:37:33,526 - root - INFO - START: Bookmarks Query
2024-07-01 15:37:33,609 - root - INFO - END: Favorites Query


(24677, 11)

Create fastai DataLoaders

In [14]:
from fastai.collab import *

dls = CollabDataLoaders.from_df(df, user_name='user', item_name='shiur', rating_name='rating', bs=64)
dls.show_batch()

,user,shiur,rating
0,76419.0,779703,1
1,222432.0,1098871,1
2,221727.0,911338,1
3,60237.0,823587,1
4,42791.0,1040452,1
5,73344.0,810334,1
6,206438.0,1065503,1
7,33951.0,957893,1
8,64504.0,1097599,1
9,218322.0,878627,1


Run the model

In [21]:
from fastai.vision.all import *

learn = collab_learner(dls, n_factors=5, y_range=(0, 1), loss_func=BCEWithLogitsLossFlat())
learn.fit_one_cycle(15, 5e-3)

epoch,train_loss,valid_loss,time
0,0.470534,0.469944,00:01
1,0.454549,0.452904,00:01
2,0.422484,0.422224,00:01
3,0.387753,0.392311,00:01
4,0.363628,0.372793,00:01
5,0.347531,0.360531,00:01
6,0.339118,0.352830,00:01
7,0.333309,0.347701,00:01
8,0.330216,0.344303,00:01
9,0.327266,0.342107,00:01


Save the model

In [22]:
learn.export("saved_models/user_collab_filtering_v1.pkl")

Use model

In [23]:
from typing import Dict, List


class UserCollabFilteringV1:
    def __init__(self):
        model = load_learner("saved_models/user_collab_filtering_v1.pkl")

    def get_recommendations(self, user_id: str = None, *args, **kwargs) -> List[int]:
        top_n = kwargs.get('top_n', 10)
        user_id = int(float(user_id))
        item_ids = dls.classes['shiur'].items[1:] # to avoid the na value
        item_ids = [int(item_id) for item_id in item_ids]
        user_tensor = torch.tensor([user_id] * len(item_ids)).unsqueeze(1)
        item_tensor = torch.tensor(item_ids).unsqueeze(1)
        input_tensor = torch.cat((user_tensor, item_tensor), dim=1)
        
        # Get predictions
        input_df = pd.DataFrame(input_tensor.numpy(), columns=['user', 'shiur'])
        user_item_dl = dls.test_dl(input_df)
        preds, _ = learn.get_preds(dl=user_item_dl)
        
        # Get top N recommendations
        top_indices = torch.argsort(preds, descending=True)[:top_n]
        top_item_ids = [item_ids[idx.item()] for idx in top_indices]
        return top_item_ids

    def get_weighted_recommendations(self, user_id: str = None, *args, **kwargs) -> Dict[int, float]:
        top_n = kwargs.get('top_n', 10)
        user_id = int(float(user_id))
        item_ids = dls.classes['shiur'].items[1:]
        item_ids = [int(item_id) for item_id in item_ids]
        user_tensor = torch.tensor([user_id] * len(item_ids)).unsqueeze(1)
        item_tensor = torch.tensor(item_ids).unsqueeze(1)
        input_tensor = torch.cat((user_tensor, item_tensor), dim=1)

        # Get predictions
        input_df = pd.DataFrame(input_tensor.numpy(), columns=['user', 'shiur'])
        user_item_dl = dls.test_dl(input_df)
        preds, _ = learn.get_preds(dl=user_item_dl)

        # Get top N recommendations with their scores
        top_indices = torch.argsort(preds, descending=True)[:top_n]
        top_item_ids = [item_ids[idx.item()] for idx in top_indices]
        top_scores = preds[top_indices].tolist()

        recommendations = {item_id: score for item_id, score in zip(top_item_ids, top_scores)}
        return recommendations

    def get_best_shiurim(self, shiur_num:int = 10): #based on highest bias
        shiur_bias = learn.model.i_bias.weight.squeeze()
        idxs = shiur_bias.argsort(descending=True)[:shiur_num]
        return [dls.classes['shiur'][i] for i in idxs]
    
    def get_user_bias(self, user_id:str = None):
        user_biases = learn.model.u_bias.weight
        user_idx = learn.dls.classes['user'].o2i[user_id]
        return user_biases[user_idx]
    
    def get_shiur_bias(self, shiur_id:str = None):
        item_biases = learn.model.i_bias.weight
        item_idx = learn.dls.classes['user'].o2i[shiur_id]
        return item_biases[item_idx]

Testing model

In [24]:
model = UserCollabFilteringV1()
print(model.get_shiur_bias('1098888'))
print(model.get_user_bias('0.387775	'))
print(model.get_best_shiurim(10))
print(model.get_recommendations("92378.0", top_n = 10))
print(model.get_weighted_recommendations("35049.0", top_n = 10))

tensor([-0.0063], grad_fn=<SelectBackward0>)
tensor([-0.0062], grad_fn=<SelectBackward0>)
[1098754, 1098495, 1098683, 1098964, 1098089, 1098679, 1098108, 1098832, 1097854, 1098342]


[1098754, 1098683, 1098964, 1098495, 1098108, 1098832, 1098089, 1098342, 1097854, 1098020]


{1098108: 0.7310585975646973, 1098683: 0.7310585975646973, 1098342: 0.7310585975646973, 1097854: 0.7310585975646973, 1098754: 0.7310585975646973, 1098832: 0.7310585975646973, 1098964: 0.7310585975646973, 1099915: 0.7310585379600525, 1098495: 0.7310585379600525, 1098574: 0.7310585379600525}
